In [ ]:
# !pip install --upgrade boto3 
# !pip install evaluate
# !pip install rouge_score

In [ ]:
import boto3
import json
import re
import pandas as pd
import evaluate
from tqdm import tqdm
from rouge_score import rouge_scorer, scoring

In [ ]:
bedrock = boto3.client(service_name="bedrock-runtime")

In [ ]:
def predict(prompt_string, context):
    body = json.dumps(
    {
        "prompt": prompt_string.format(context),
        "max_tokens_to_sample": 500,
    }
    )

    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-v2")    
    response_body = json.loads(response.get("body").read())
    impression = re.findall("<impression>(.*?)</impression>", response_body.get("completion"), re.DOTALL)
    # print(response_body.get("completion"))
    if impression == []:
        return response_body.get("completion")
    return impression[0]

In [ ]:
def generate_impressions(prompt, filename):
    dev_df = pd.read_csv(filename)
    print(f"num rows: {len(dev_df)}")
    
    output_bedrock = []
    for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        generated_impression = predict(prompt, row['findings'])
        # print(f"Predicted: {generated_impression}, \nGT: {row['impression']}")
        output_bedrock.append(generated_impression)
        # break
    
    dev_df['claude-v2-generated-impressions'] = output_bedrock    
    return dev_df

In [ ]:
def calculate_rouge_scores(dev_df, prompt, filename):
    print('PROMPT used:\n', prompt)
    print("-"*50)
    rouge_score = evaluate.load("rouge") 
    result_pretrained_dev1 = rouge_score.compute(predictions=list(dev_df["claude-v2-generated-impressions"]), references=list(dev_df["impression"]))
    print("ROUGE Score for claude-v2 model on ", filename)
    print(result_pretrained_dev1)
    print("-"*50)

## Zero shot prompting

In [ ]:
# prompt_zero_shot = f"""Human: Generate a new short impression from the following Radiology findings section using medical vocabulary and output it within <impression> tags. Findings: {context}

# Assistant:"""

prompt_zero_shot = """Human: Generate radiology report impressions based on the following findings and output it within <impression> tags. Findings: {}

Assistant:"""

#### generate impressions for dev1_MIMICXR.csv

In [ ]:
filename='dev1_MIMICXR.csv'

dev1_df_zero_shot = generate_impressions(prompt_zero_shot, filename)

##### ROUGE Score Computation for dev1_MIMICXR.csv

In [ ]:
calculate_rouge_scores(dev1_df_zero_shot, prompt=prompt_zero_shot, filename='dev1_MIMICXR.csv')

#### generate impressions for dev2_Indiana.csv

In [ ]:
filename='dev2_Indiana.csv'

dev2_df_zero_shot = generate_impressions(prompt_zero_shot, filename)

##### ROUGE Score Computation for dev2_Indiana.csv

In [ ]:
calculate_rouge_scores(dev2_df_zero_shot, prompt=prompt_zero_shot, filename='dev2_Indiana.csv')

## Few-shot prompting

In [ ]:
examples = [
    {
        "findings": 'Heart size is normal. No focal airspace consolidations. No pneumothorax or effusion. No acute osseous findings.',
        "impression": 'No acute cardiopulmonary findings'
    }, 
    {
        "findings": 'The lungs are clear. There is no pleural effusion or pneumothorax. There is no evidence for mass lung apices. The heart is normal. There are atherosclerotic changes of the aorta. The skeletal structures are unremarkable.',
        "impression": 'No acute pulmonary disease. No evidence for apical mass'
    }, 
    {
        "findings": 'The lungs are clear, and without focal airspace opacity. The cardiomediastinal silhouette is normal in size and contour, and stable. There is no pneumothorax or large pleural effusion.',
        "impression": 'No acute cardiopulmonary abnormality'
    }, 
    {
        "findings": 'Heart size and pulmonary vascularity appear within normal limits. Lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen.',
        "impression": 'No evidence of active disease'
    }, 
    {
        "findings": 'The lungs appear clear. The heart and pulmonary ____ are normal. The pleural spaces are clear. Mediastinal contours are normal.',
        "impression": 'No acute acute cardiopulmonary disease.'
    }
]

examples_string = ''
for ex in examples:
    examples_string += f"""H:<findings>{ex['findings']}</findings>
    
A:<impression>{ex['impression']}</impression>\n"""
    
    
prompt_few_shot = """Human: Generate radiology report impressions based on the following findings. Findings: 
<findings>{}</findings>
        
Here are a few examples:
<examples>
""" + examples_string + """
</examples>

Assistant:"""

#### generate impressions for dev1_MIMICXR.csv

In [ ]:
filename='dev1_MIMICXR.csv'

dev1_df_few_shot = generate_impressions(prompt_few_shot, filename)

##### ROUGE Score Computation for dev1_MIMICXR.csv

In [ ]:
calculate_rouge_scores(dev1_df_few_shot, prompt=prompt_few_shot, filename='dev1_MIMICXR.csv')

#### generate impressions for dev1_MIMICXR.csv

In [ ]:
filename='dev2_Indiana.csv'

dev2_df_few_shot = generate_impressions(prompt_few_shot, filename)

##### ROUGE Score Computation for dev1_MIMICXR.csv

In [ ]:
calculate_rouge_scores(dev2_df_few_shot, prompt=prompt_few_shot, filename='dev2_Indiana.csv')